In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Normalize the pixel values between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert the labels to one-hot encoding
y_train = keras.utils.to_categorical(y_train, 100)
y_test = keras.utils.to_categorical(y_test, 100)


In [2]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import os
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
import glob

target_size = (32, 32)  # Change the values as per your requirement

# Load the pre-trained ResNet50 model with modified input shape
model = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(target_size[0], target_size[1], 3))

In [3]:
import pandas as pd

# Define the file path
file_path = r"D:\feature.csv"

# Read the tab-separated CSV file into a DataFrame
df = pd.read_csv(file_path, delimiter='\t')

columns_to_drop = [df.columns[0], df.columns[-1]]
data = df.drop(columns_to_drop, axis=1)
# Display the head of the DataFrame
print(data.head(1))


   Feature 0  Feature 1  Feature 2  Feature 3  Feature 4  Feature 5  \
0        0.0        0.0        0.0        0.0        0.0        0.0   

   Feature 6  Feature 7  Feature 8  Feature 9  ...  Feature 2038  \
0        0.0        0.0        0.0        0.0  ...           0.0   

   Feature 2039  Feature 2040  Feature 2041  Feature 2042  Feature 2043  \
0           0.0      0.009121           0.0           0.0           0.0   

   Feature 2044  Feature 2045  Feature 2046  Feature 2047  
0           0.0      3.285227           0.0           0.0  

[1 rows x 2048 columns]


In [4]:
from PIL import Image
import numpy as np
ft = model.predict(x_train)


In [11]:
from sklearn.cluster import MiniBatchKMeans

n_clusters = 200
batch_size = 100
max_iter = 100

kmeans = MiniBatchKMeans(n_clusters=n_clusters, batch_size=batch_size, max_iter=max_iter)
kmeans.fit(ft)
# Retrieve the cluster centers
ct = kmeans.cluster_centers_.tolist()

C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\sklearn\cluster\_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


In [14]:
from tqdm import tqdm

combo_list = []  # Initialize combo_list

# Iterate over ft
for i in tqdm(range(len(ct))):
    distances = []

    # Calculate distances for each row in data
    for index, row in data.iterrows():
        row_array = row.to_numpy()  # Convert row to numpy array
        distance = np.linalg.norm(ct[i] - row_array)  # Calculate Euclidean distance
        distances.append(distance)

    # Retrieve shortest indices and select corresponding rows from df
    shortest_indices = np.argsort(distances)[:201]
    selected_rows = df.loc[shortest_indices]

    # Extract values from the first column and add them to the combo_list
    first_column_values = selected_rows.iloc[:, 0].tolist()
    
    k = 0
    
    for element in first_column_values:
        if element not in combo_list:
            combo_list.append(element)
            break

# Printing the combo_list
print(combo_list)



  0%|                                                                                          | 0/200 [00:00<?, ?it/s]

  0%|▍                                                                                 | 1/200 [00:00<00:40,  4.94it/s]

  1%|▊                                                                                 | 2/200 [00:00<00:41,  4.76it/s]

  2%|█▏                                                                                | 3/200 [00:00<00:40,  4.90it/s]

  2%|█▋                                                                                | 4/200 [00:00<00:39,  4.94it/s]

  2%|██                                                                                | 5/200 [00:01<00:38,  5.04it/s]

  3%|██▍                                                                               | 6/200 [00:01<00:38,  5.10it/s]

  4%|██▊                                                                               | 7/200 [00:01<00:39,  4.83it/s]

  4%|███▎                     

 32%|█████████████████████████▉                                                       | 64/200 [00:13<00:30,  4.51it/s]

 32%|██████████████████████████▎                                                      | 65/200 [00:14<00:30,  4.47it/s]

 33%|██████████████████████████▋                                                      | 66/200 [00:14<00:29,  4.53it/s]

 34%|███████████████████████████▏                                                     | 67/200 [00:14<00:29,  4.44it/s]

 34%|███████████████████████████▌                                                     | 68/200 [00:14<00:32,  4.08it/s]

 34%|███████████████████████████▉                                                     | 69/200 [00:15<00:30,  4.35it/s]

 35%|████████████████████████████▎                                                    | 70/200 [00:15<00:30,  4.22it/s]

 36%|████████████████████████████▊                                                    | 71/200 [00:15<00:32,  3.99it/s]

 36%|███████████████████████████

 64%|███████████████████████████████████████████████████▏                            | 128/200 [00:28<00:15,  4.66it/s]

 64%|███████████████████████████████████████████████████▌                            | 129/200 [00:28<00:15,  4.57it/s]

 65%|████████████████████████████████████████████████████                            | 130/200 [00:28<00:16,  4.31it/s]

 66%|████████████████████████████████████████████████████▍                           | 131/200 [00:28<00:16,  4.10it/s]

 66%|████████████████████████████████████████████████████▊                           | 132/200 [00:29<00:16,  4.24it/s]

 66%|█████████████████████████████████████████████████████▏                          | 133/200 [00:29<00:15,  4.38it/s]

 67%|█████████████████████████████████████████████████████▌                          | 134/200 [00:29<00:14,  4.57it/s]

 68%|██████████████████████████████████████████████████████                          | 135/200 [00:29<00:14,  4.61it/s]

 68%|███████████████████████████

 96%|████████████████████████████████████████████████████████████████████████████▊   | 192/200 [00:42<00:01,  4.39it/s]

 96%|█████████████████████████████████████████████████████████████████████████████▏  | 193/200 [00:43<00:01,  4.60it/s]

 97%|█████████████████████████████████████████████████████████████████████████████▌  | 194/200 [00:43<00:01,  4.63it/s]

 98%|██████████████████████████████████████████████████████████████████████████████  | 195/200 [00:43<00:01,  4.80it/s]

 98%|██████████████████████████████████████████████████████████████████████████████▍ | 196/200 [00:43<00:00,  4.26it/s]

 98%|██████████████████████████████████████████████████████████████████████████████▊ | 197/200 [00:44<00:00,  4.46it/s]

 99%|███████████████████████████████████████████████████████████████████████████████▏| 198/200 [00:44<00:00,  4.37it/s]

100%|███████████████████████████████████████████████████████████████████████████████▌| 199/200 [00:44<00:00,  4.49it/s]

100%|███████████████████████████

['D:/data/imagenet\\n02009912\\', 'D:/data/imagenet\\n07836838\\', 'D:/data/imagenet\\n07802026\\', 'D:/data/imagenet\\n10148035\\', 'D:/data/imagenet\\n03393912\\', 'D:/data/imagenet\\n01828970\\', 'D:/data/imagenet\\n09468604\\', 'D:/data/imagenet\\n04146614\\', 'D:/data/imagenet\\n03706229\\', 'D:/data/imagenet\\n03207743\\', 'D:/data/imagenet\\n09421951\\', 'D:/data/imagenet\\n04525305\\', 'D:/data/imagenet\\n03692522\\', 'D:/data/imagenet\\n04019541\\', 'D:/data/imagenet\\n03495258\\', 'D:/data/imagenet\\n03016953\\', 'D:/data/imagenet\\n07579787\\', 'D:/data/imagenet\\n03297495\\', 'D:/data/imagenet\\n11879895\\', 'D:/data/imagenet\\n09288635\\', 'D:/data/imagenet\\n11939491\\', 'D:/data/imagenet\\n03773504\\', 'D:/data/imagenet\\n02510455\\', 'D:/data/imagenet\\n03216828\\', 'D:/data/imagenet\\n04392985\\', 'D:/data/imagenet\\n03197337\\', 'D:/data/imagenet\\n02667093\\', 'D:/data/imagenet\\n04552348\\', 'D:/data/imagenet\\n02018795\\', 'D:/data/imagenet\\n04523525\\', 'D:/data/

In [18]:
import pickle

# Saving the list to a file in the D: drive
file_path = r"D:\combo_list.pickle"  # r prefix is used for raw string to avoid escape characters

with open(file_path, 'wb') as file:
    pickle.dump(combo_list, file)

print("Combo list saved successfully.")


Combo list saved successfully.


In [19]:
import pickle

# Loading the list from the pickle file
file_path = r"D:\combo_list.pickle"  # Update the file path if necessary

with open(file_path, 'rb') as file:
    loaded_combo_list = pickle.load(file)

print("Combo list loaded successfully.")
print(loaded_combo_list)


Combo list loaded successfully.
['D:/data/imagenet\\n02009912\\', 'D:/data/imagenet\\n07836838\\', 'D:/data/imagenet\\n07802026\\', 'D:/data/imagenet\\n10148035\\', 'D:/data/imagenet\\n03393912\\', 'D:/data/imagenet\\n01828970\\', 'D:/data/imagenet\\n09468604\\', 'D:/data/imagenet\\n04146614\\', 'D:/data/imagenet\\n03706229\\', 'D:/data/imagenet\\n03207743\\', 'D:/data/imagenet\\n09421951\\', 'D:/data/imagenet\\n04525305\\', 'D:/data/imagenet\\n03692522\\', 'D:/data/imagenet\\n04019541\\', 'D:/data/imagenet\\n03495258\\', 'D:/data/imagenet\\n03016953\\', 'D:/data/imagenet\\n07579787\\', 'D:/data/imagenet\\n03297495\\', 'D:/data/imagenet\\n11879895\\', 'D:/data/imagenet\\n09288635\\', 'D:/data/imagenet\\n11939491\\', 'D:/data/imagenet\\n03773504\\', 'D:/data/imagenet\\n02510455\\', 'D:/data/imagenet\\n03216828\\', 'D:/data/imagenet\\n04392985\\', 'D:/data/imagenet\\n03197337\\', 'D:/data/imagenet\\n02667093\\', 'D:/data/imagenet\\n04552348\\', 'D:/data/imagenet\\n02018795\\', 'D:/data/i